# Raw Data Processing

## HS 300/Inno 508 Financial Statistics Data Generation (Quarterly)

In [1]:
import pandas as pd
df = pd.read_excel('./data/inno_stock/finance_statistics_01.xlsx')


In [2]:
df.head(150)

,Stkcd,Accper,Typrep,A001101000,A0d1101101,A0d1102000,A0d1102101,A0b1103000,A0b1104000,A0b1105000,...,A001228000,A001229000,A002128000,A001230000,A002211000,IfCorrect,DeclareDate,ShortName,A0F1224000,A0F2210000
0,证券代码,统计截止日期,报表类型,货币资金,其中:客户资金存款,结算备付金,其中：客户备付金,现金及存放中央银行款项,存放同业款项,贵金属,...,其他权益工具投资,其他非流动金融资产,合同负债,使用权资产,租赁负债,是否发生差错更正,差错更正披露日期,证券简称,代理业务资产,代理业务负债
1,没有单位,没有单位,没有单位,元,元,元,元,元,元,元,...,元,NaN,元,元,元,没有单位,没有单位,没有单位,元,元
2,000001,2019-01-01,A,NaN,NaN,NaN,NaN,278528000000,85098000000,56835000000,...,1519000000,NaN,NaN,NaN,NaN,1,2020-02-14,平安银行,NaN,NaN
3,000001,2019-03-31,A,NaN,NaN,NaN,NaN,246856000000,69911000000,54173000000,...,1565000000,NaN,NaN,NaN,NaN,0,NaN,平安银行,NaN,NaN
4,000001,2019-06-30,A,NaN,NaN,NaN,NaN,269794000000,97580000000,65624000000,...,1816000000,NaN,NaN,NaN,NaN,0,NaN,平安银行,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,000006,2020-06-30,B,517739692.58,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,749968.6,NaN,NaN,0,NaN,深振业A,NaN,NaN
146,000006,2020-09-30,A,2900411088.59,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1750839946.39,NaN,NaN,0,NaN,深振业A,NaN,NaN
147,000006,2020-09-30,B,981426860.32,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2038145.73,NaN,NaN,0,NaN,深振业A,NaN,NaN
148,000006,2020-12-31,A,1947760245.47,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1127186548.96,NaN,NaN,0,NaN,深振业A,NaN,NaN


In [3]:
com_attr_cols = ['Stkcd', 'Accper', 'Typrep', 'A001000000', 'A002000000', 'A001219000'] # total assets, total liability, R&D


In [4]:
hs_300_com = []
# with open('./data/inno_stock/hs300_code', 'r') as f:
with open('./data/inno_stock/inno_508_code', 'r') as f:
    lines = f.readlines()
    for line in lines:
        hs_300_com.append(line.strip())

In [5]:
len(list(set(hs_300_com)))

507

In [6]:
quaterly_date = ['2020-03-31', '2020-06-30', '2020-09-30', '2020-12-31', '2021-03-31', '2021-06-30', '2021-09-30', '2021-12-31', '2022-03-31', '2022-06-30', '2022-09-30', '2022-12-31']


In [8]:
df_small = df[com_attr_cols]
df_300 = df_small[df_small['Stkcd'].isin(hs_300_com)]
df_300_latest = df_300[df_300['Accper'].isin(quaterly_date)]
df_300_latest = df_300_latest[df_300_latest['Typrep'] == 'A']
df_300_latest = df_300_latest.fillna(0)

In [9]:
df_300_latest

,Stkcd,Accper,Typrep,A001000000,A002000000,A001219000
132673,688001,2020-03-31,A,2.142980e+09,2.363236e+08,0.0
132675,688001,2020-06-30,A,3.524663e+09,8.380054e+08,0.0
132677,688001,2020-09-30,A,3.603285e+09,8.164610e+08,0.0
132679,688001,2020-12-31,A,3.645404e+09,4.775211e+08,0.0
132683,688001,2021-03-31,A,3.747761e+09,5.437248e+08,0.0
...,...,...,...,...,...,...
139970,689009,2021-09-30,A,7.576372e+09,3.395097e+09,0.0
139971,689009,2021-12-31,A,7.672446e+09,3.399555e+09,0.0
139972,689009,2022-03-31,A,7.670039e+09,3.334013e+09,0.0
139973,689009,2022-06-30,A,8.988434e+09,4.391805e+09,0.0


In [10]:
df_out = df_300_latest[['Stkcd', 'A001000000', 'A002000000', 'A001219000', 'Accper']]
# hs_300_stats = df_out.set_index('Stkcd').T.to_dict('list')
hs_300_stats = df_out.values.tolist()

In [11]:
hs_300_stats

[['688001', 2142980093.58, 236323557.62, 0.0, '2020-03-31'],
 ['688001', 3524663446.93, 838005418.16, 0.0, '2020-06-30'],
 ['688001', 3603285172.04, 816461044.25, 0.0, '2020-09-30'],
 ['688001', 3645404409.88, 477521141.22, 0.0, '2020-12-31'],
 ['688001', 3747761415.94, 543724756.39, 0.0, '2021-03-31'],
 ['688001', 4064270355.4, 819461246.27, 0.0, '2021-06-30'],
 ['688001', 4244946585.45, 847353707.47, 0.0, '2021-09-30'],
 ['688001', 5150193613.8, 1618455597.93, 0.0, '2021-12-31'],
 ['688001', 5115562056.88, 1537162921.88, 0.0, '2022-03-31'],
 ['688001', 5347519808.29, 1629082863.12, 0.0, '2022-06-30'],
 ['688001', 5382014033.73, 1607371108.41, 0.0, '2022-09-30'],
 ['688002', 2763881207.17, 336249325.06, 0.0, '2020-03-31'],
 ['688002', 3084075591.31, 453504922.34, 0.0, '2020-06-30'],
 ['688002', 3281079251.71, 493962127.79, 0.0, '2020-09-30'],
 ['688002', 3544575592.74, 614810188.85, 0.0, '2020-12-31'],
 ['688002', 3711381439.18, 609718381.64, 0.0, '2021-03-31'],
 ['688002', 3979614726

In [12]:
import pickle

# with open('./data/inno_stock/hs_300_stats_quarterly.pkl', 'wb') as f:
with open('./data/inno_stock/inno_508_stats_quarterly.pkl', 'wb') as f:
    pickle.dump(hs_300_stats, f)

In [58]:
with open('./data/inno_stock/hs_300_stats_quarterly.pkl', 'rb') as f:
    tmp = pickle.load(f)

In [59]:
tmp

[['000001', 4132298000000.0, 3779943000000.0, 0.0, 0.0, '2020-03-31'],
 ['000001', 4178622000000.0, 3827225000000.0, 0.0, 0.0, '2020-06-30'],
 ['000001', 4346921000000.0, 3988150000000.0, 0.0, 0.0, '2020-09-30'],
 ['000001', 4468514000000.0, 4104383000000.0, 0.0, 0.0, '2020-12-31'],
 ['000001', 4572974000000.0, 4200357000000.0, 0.0, 0.0, '2021-03-31'],
 ['000001', 4723520000000.0, 4346327000000.0, 0.0, 0.0, '2021-06-30'],
 ['000001', 4853074000000.0, 4464216000000.0, 0.0, 0.0, '2021-09-30'],
 ['000001', 4921380000000.0, 4525932000000.0, 0.0, 0.0, '2021-12-31'],
 ['000001', 5117949000000.0, 4711774000000.0, 0.0, 0.0, '2022-03-31'],
 ['000001', 5108776000000.0, 4696678000000.0, 0.0, 0.0, '2022-06-30'],
 ['000001', 5195133000000.0, 4769749000000.0, 0.0, 0.0, '2022-09-30'],
 ['000002',
  1739794796342.85,
  1465683669930.9,
  0.0,
  2627512620.48,
  '2020-03-31'],
 ['000002',
  1806186613769.1,
  1514930506717.89,
  0.0,
  2640266479.94,
  '2020-06-30'],
 ['000002',
  1814465741941.72,
  1

## HS 300 Industry Class Data Generation

In [13]:
hs_300_com = []
# with open('./data/inno_stock/hs300_code', 'r') as f:
with open('./data/inno_stock/inno_508_code', 'r') as f:
    lines = f.readlines()
    for line in lines:
        hs_300_com.append(line.strip())

In [14]:
hs_300_com[:10]

['688012',
 '688111',
 '688063',
 '688223',
 '688599',
 '688297',
 '688387',
 '688787',
 '688041',
 '688039']

In [15]:
import pandas as pd
df = pd.read_excel('./data/inno_stock/industry_class_01.xlsx')

In [18]:
df.head(20)

,InstitutionID,Symbol,IndustryClassificationID,IndustryClassificationName,ImplementDate,IndustryCode,IndustryName
0,上市公司ID,股票代码,行业分类标准编码,行业分类标准,实施开始日期,行业代码,行业名称
1,没有单位,没有单位,没有单位,没有单位,没有单位,没有单位,没有单位
2,101704,000001,P0211,申银万国行业分类2014修订版,1991-04-03,480101,银行
3,101704,000001,P0207,证监会行业分类2012年版,1991-04-03,J66,货币金融服务
4,101704,000001,P0201,证监会行业分类2001年版,1991-04-03,I0199,其他商业银行
5,101704,000001,P0203,申银万国行业分类2011修订版,1991-04-03,440101,银行
6,101704,000001,P0206,中证行业分类,2012-05-16,06010101,综合性银行
7,101704,000001,P0201,证监会行业分类2001年版,2012-10-29,I01,银行业
8,101704,000001,P0212,中信证券行业分类,2015-08-11,ZX310201,股份制银行
9,101704,000001,P0215,中证行业分类2016年版,2016-12-12,06010101,综合性银行


In [17]:
com_attr_cols = ['Symbol', 'IndustryCode', 'IndustryName', 'IndustryClassificationName']

In [19]:
df_small = df[com_attr_cols]
df_300 = df_small[df_small['Symbol'].isin(hs_300_com)]
df_300_sw = df_300[df_300['IndustryClassificationName'] == '申银万国行业分类2021修订版']

In [20]:
df_300_sw

,Symbol,IndustryCode,IndustryName,IndustryClassificationName
46000,688001,640107,仪器仪表,申银万国行业分类2021修订版
46008,688002,650501,军工电子Ⅲ,申银万国行业分类2021修订版
46015,688003,640704,其他自动化设备,申银万国行业分类2021修订版
46021,688004,710301,IT服务Ⅲ,申银万国行业分类2021修订版
46028,688005,630702,电池化学品,申银万国行业分类2021修订版
...,...,...,...,...
48820,688799,370102,化学制剂,申银万国行业分类2021修订版
48826,688800,270401,其他电子Ⅲ,申银万国行业分类2021修订版
48832,688819,630705,蓄电池及其他电池,申银万国行业分类2021修订版
48839,688981,270106,集成电路制造,申银万国行业分类2021修订版


In [21]:
df_out = df_300_sw[['Symbol', 'IndustryCode', 'IndustryName']]
hs_300_industry = df_out.values.tolist()

In [22]:
hs_300_industry

[['688001', '640107', '仪器仪表'],
 ['688002', '650501', '军工电子Ⅲ'],
 ['688003', '640704', '其他自动化设备'],
 ['688004', '710301', 'IT服务Ⅲ'],
 ['688005', '630702', '电池化学品'],
 ['688006', '630703', '锂电专用设备'],
 ['688007', '270503', '品牌消费电子'],
 ['688008', '270104', '数字芯片设计'],
 ['688009', '640501', '轨交设备Ⅲ'],
 ['688010', '270303', '光学元件'],
 ['688011', '650501', '军工电子Ⅲ'],
 ['688012', '270108', '半导体设备'],
 ['688013', '370503', '医疗耗材'],
 ['688015', '640501', '轨交设备Ⅲ'],
 ['688016', '370503', '医疗耗材'],
 ['688017', '640701', '机器人'],
 ['688018', '270104', '数字芯片设计'],
 ['688019', '270601', '电子化学品Ⅲ'],
 ['688020', '270202', '印制电路板'],
 ['688021', '280205', '其他汽车零部件'],
 ['688022', '640209', '其他专用设备'],
 ['688023', '710402', '横向通用软件'],
 ['688025', '640703', '激光设备'],
 ['688026', '220501', '其他塑料制品'],
 ['688026', '370503', '医疗耗材'],
 ['688027', '730206', '通信终端及配件'],
 ['688028', '640103', '磨具磨料'],
 ['688029', '370503', '医疗耗材'],
 ['688030', '710402', '横向通用软件'],
 ['688031', '710402', '横向通用软件'],
 ['688032', '630503', '逆变器'],
 ['6

In [23]:
import pickle

# with open('./data/inno_stock/hs_300_industry.pkl', 'wb') as f:
with open('./data/inno_stock/inno_508_industry.pkl', 'wb') as f:
    pickle.dump(hs_300_industry, f)

## HS 300 Supply Chain Data Generation

In [24]:
hs_300_com = []
# with open('./data/inno_stock/hs300_code', 'r') as f:
with open('./data/inno_stock/inno_508_code', 'r') as f:
    lines = f.readlines()
    for line in lines:
        hs_300_com.append(line.strip())

In [25]:
import pandas as pd
df = pd.read_excel('./data/inno_stock/supply_relation_01.xlsx')

/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [31]:
com_attr_cols = ['Symbol', 'EndDate', 'StateTypeCode', 'PSCBusinessRelation', 'PSCSymbol', 'SSCBusinessRelation', 'SSCSymbol']

In [32]:
df_small = df[com_attr_cols]
df_300 = df_small[df_small['Symbol'].isin(hs_300_com)]
df_300_merge = df_300[df_300['StateTypeCode'] == 1]

In [33]:
df_300_merge

,Symbol,EndDate,StateTypeCode,PSCBusinessRelation,PSCSymbol,SSCBusinessRelation,SSCSymbol
8166,688086,2020-12-31,1,2,836769,NaN,NaN
8167,688086,2021-12-31,1,1,600536,NaN,NaN
8168,688086,2021-12-31,1,1,603927,NaN,NaN
8169,688086,2021-12-31,1,2,000066,NaN,NaN
8170,688148,2021-12-31,1,1,300073,NaN,NaN
...,...,...,...,...,...,...,...
8241,688779,2021-12-31,1,2,002497,NaN,NaN
8242,688788,2020-12-31,1,2,002179,NaN,NaN
8243,688788,2021-12-31,1,2,002179,NaN,NaN
8244,688799,2021-12-31,1,1,000650,NaN,NaN


In [29]:
# df_300_merge = df_300_merge[df_300_merge['PSCSymbol'].isin(hs_300_com)]

In [30]:
# df_300_merge

,Symbol,EndDate,StateTypeCode,PSCBusinessRelation,PSCSymbol,SSCBusinessRelation,SSCSymbol
8175,688171,2021-12-31,1,2,688159,NaN,NaN
8234,688598,2020-12-31,1,2,688295,NaN,NaN
8237,688737,2021-12-31,1,2,688021,NaN,NaN
8238,688766,2021-12-31,1,2,688372,NaN,NaN
8240,688779,2021-12-31,1,1,688567,NaN,NaN


In [34]:
df_out = df_300_merge[['Symbol', 'PSCBusinessRelation', 'PSCSymbol']]
hs_300_supply = df_out.values.tolist()

In [35]:
hs_300_supply

[['688086', 2, '836769'],
 ['688086', 1, '600536'],
 ['688086', 1, '603927'],
 ['688086', 2, '000066'],
 ['688148', 1, '300073'],
 ['688148', 1, '835185'],
 ['688148', 1, '835930'],
 ['688169', 1, 'JD'],
 ['688169', 2, '300207'],
 ['688171', 2, '688159'],
 ['688171', 1, '600756'],
 ['688189', 2, '834804'],
 ['688189', 2, '600529'],
 ['688199', 2, '301118'],
 ['688217', 2, '834618'],
 ['688285', 2, '300697'],
 ['688285', 2, '600219'],
 ['688296', 2, '300014'],
 ['688321', 1, '01099'],
 ['688321', 1, '02607'],
 ['688321', 2, '836817'],
 ['688321', 1, '601607'],
 ['688321', 1, '601607'],
 ['688321', 1, '01099'],
 ['688321', 1, '02607'],
 ['688321', 1, '01099'],
 ['688321', 1, '02607'],
 ['688321', 1, '601607'],
 ['688329', 1, '01099'],
 ['688329', 1, '02607'],
 ['688329', 1, '03320'],
 ['688329', 1, '600713'],
 ['688329', 1, '601607'],
 ['688330', 1, '839644'],
 ['688330', 1, '839644'],
 ['688336', 1, '000963'],
 ['688336', 1, '01099'],
 ['688336', 1, '02607'],
 ['688336', 1, '03320'],
 [

In [36]:
hs_300_supply_out = []
for item in hs_300_supply:
    if item[1] == 2:
        hs_300_supply_out.append([item[2], item[0]])
    else:
        hs_300_supply_out.append([item[0], item[2]])

In [37]:
import pickle

# with open('./data/inno_stock/hs_300_supply.pkl', 'wb') as f:
with open('./data/inno_stock/inno_508_supply.pkl', 'wb') as f:
    pickle.dump(hs_300_supply_out, f)

## HS 300 Investment Data Generation

In [38]:
hs_300_com = []
# with open('./data/inno_stock/hs300_code', 'r') as f:
with open('./data/inno_stock/inno_508_code', 'r') as f:
    lines = f.readlines()
    for line in lines:
        hs_300_com.append(line.strip())

In [39]:
import pandas as pd
df = pd.read_excel('./data/inno_stock/investment_info_01.xlsx')

In [40]:
com_attr_cols = ['BusinessSymbol', 'CompanyName', 'Investment', 'InvestRatio']

In [41]:
df_small = df[com_attr_cols]
df_300 = df_small[df_small['BusinessSymbol'].isin(hs_300_com)]
df_300 = df_300.fillna(0)

In [42]:
df_300

,BusinessSymbol,CompanyName,Investment,InvestRatio
4345,688579,衢州地纬软件有限责任公司,50.0,100.0
4346,688579,山大地纬软件股份有限公司深圳分公司,0.0,100.0
4347,688579,山东英佰德信息科技有限公司,240.0,100.0
4348,688579,北京地纬赛博科技有限公司,500.0,100.0
4349,688579,山东山大地纬健康科技有限公司,750.0,75.0
...,...,...,...,...
226530,688669,常州奥智高分子集团股份有限公司,2677.5,51.0
226531,688669,安徽聚石科技有限公司,5000.0,100.0
226532,688669,安庆聚信新材料科技有限公司,9000.0,90.0
226889,688767,杭州博旺生物科技有限公司,1000.0,100.0


In [43]:
df_out = df_300[['BusinessSymbol', 'CompanyName', 'Investment', 'InvestRatio']]
hs_300_invest = df_out.values.tolist()

In [44]:
hs_300_invest_out = []
for item in hs_300_invest:
    if item in hs_300_invest_out:
        continue
    else:
        hs_300_invest_out.append(item)

In [45]:
len(hs_300_invest_out)

856

In [46]:
investor_id = {}
iid = 0
for item in hs_300_invest_out:
    if item[1] not in investor_id:
        investor_id[item[1]] = iid
        iid += 1
    else:
        continue


In [47]:
len(list(investor_id.keys()))

821

In [50]:
hs_300_invest_final = []
for item in hs_300_invest_out:
    hs_300_invest_final.append([item[0], str(investor_id[item[1]]), item[2], item[3]])

In [51]:
hs_300_invest_final

[['688579', '0', 50.0, 100.0],
 ['688579', '1', 0.0, 100.0],
 ['688579', '2', 240.0, 100.0],
 ['688579', '3', 500.0, 100.0],
 ['688579', '4', 750.0, 75.0],
 ['688579', '5', 450.0, 45.0],
 ['688579', '6', 7500.0, 3.0],
 ['688133', '7', 200.0, 100.0],
 ['688133', '8', 100.0, 100.0],
 ['688133', '9', 200.0, 100.0],
 ['688133', '10', 3000.0, 100.0],
 ['688133', '11', 500.0, 100.0],
 ['688133', '12', 850.0, 100.0],
 ['688133', '13', 500.0, 100.0],
 ['688133', '14', 200.0, 100.0],
 ['688133', '15', 1200.0, 100.0],
 ['688133', '16', 75.0, 75.0],
 ['688133', '17', 1950.0, 65.0],
 ['688133', '18', 65.0, 65.0],
 ['688599', '19', 2500.0, 100.0],
 ['688599', '20', 1000.0, 100.0],
 ['688599', '21', 100.0, 100.0],
 ['688599', '22', 9500.0, 100.0],
 ['688599', '23', 60000.0, 100.0],
 ['688599', '24', 11850.0, 100.0],
 ['688599', '25', 4480.0, 100.0],
 ['688599', '26', 5000.0, 100.0],
 ['688599', '27', 118956.81, 100.0],
 ['688599', '28', 61373.6, 100.0],
 ['688599', '29', 3000.0, 100.0],
 ['688599', 

In [52]:
import pickle

# with open('./data/inno_stock/hs_300_invest.pkl', 'wb') as f:
with open('./data/inno_stock/inno_508_invest.pkl', 'wb') as f:
    pickle.dump(hs_300_invest_final, f)

## HS 300 Stock Price Data Generation

In [10]:
hs_300_com = []
# with open('./data/inno_stock/hs300_code', 'r') as f:
with open('./data/inno_stock/inno_508_code', 'r') as f:
    lines = f.readlines()
    for line in lines:
        hs_300_com.append(line.strip())

In [57]:
import pandas as pd

hs_300_price_out = []
parts = ['01','02','03','04']
price_attr_cols = ['Stkcd', 'Trddt', 'Clsprc', 'Dretwd']
for p in parts:
    df = pd.read_excel('./data/inno_stock/stock_trade_daily_'+p+'.xlsx')
    df_small = df[price_attr_cols]
    df_300 = df_small[df_small['Stkcd'].isin(hs_300_com)]
    df_300 = df_300.fillna(0)
    hs_300_price = df_300.values.tolist()
    hs_300_price_out.extend(hs_300_price)
    

/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [58]:
len(hs_300_price_out)

210512

In [59]:
hs_300_price_out[:5]

[['688001', '2020-01-02', 46.05, 0.039973],
 ['688001', '2020-01-03', 47.3, 0.027144],
 ['688001', '2020-01-06', 46.76, -0.011416],
 ['688001', '2020-01-07', 47.8, 0.022241],
 ['688001', '2020-01-08', 46.6, -0.025105]]

In [60]:
import pickle
# with open('./data/inno_stock/hs_300_price.pkl', 'wb') as f:
with open('./data/inno_stock/inno_508_price.pkl', 'wb') as f:
    pickle.dump(hs_300_price_out, f)

In [100]:
tmp = []
for item in hs_300_price_out:
    if item[1] == '2022-01-04':
        tmp.append(item[0])

In [101]:
len(tmp)

376

## HS 300 Stock News Data Generation

In [8]:
hs_300_com = []
# with open('./data/inno_stock/hs300_code', 'r') as f:
with open('./data/inno_stock/inno_508_code', 'r') as f:
    lines = f.readlines()
    for line in lines:
        hs_300_com.append(line.strip())

In [9]:
import pandas as pd
df = pd.read_excel('./data/inno_stock/news_stock_link.xlsx')

/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [113]:
link_attr_cols = ['NewsID', 'DeclareDate', 'Title', 'Symbol']
df_small = df[link_attr_cols]
df_300 = df_small[df_small['Symbol'].isin(hs_300_com)]
df_300 = df_300.fillna(0)

In [114]:
df_300

,NewsID,DeclareDate,Title,Symbol
98119,104597500,2020-07-01,“焊接专家”联赢激光（688518）：重视研发 技术与市场优势显著,688518
98123,104597749,2020-07-01,天合光能拟与TPG集团签署约7亿美元合同,688599
98135,104598568,2020-07-01,四川第二家科创板企业上市交易 秦川物联首日盘中涨逾170%,688528
98136,104598603,2020-07-01,博瑞医药收监管工作函股价下挫 今年来4收监管文书,688166
98137,104598876,2020-07-01,西安“市长特别奖”名单出炉 西部超导等多家上市公司入选,688122
...,...,...,...,...
586097,106492900,2022-12-31,和元生物上海临港精准医疗产业基地将于2023年第一季度投入试运行,688238
586102,106493111,2022-12-31,从技术突破到场景落地 开普云元宇宙战略不断深化,688228
586104,106493269,2022-12-31,天风证券给予禾川科技买入评级：国产伺服系统引领者 产业链上下游联动助力,688320
586106,106493387,2022-12-31,立案调查！公司、董事长、实控人、董秘、财务总监集体被罚,688656


In [117]:
# df_300_new = df_300[df_300['DeclareDate'] >= '2022-01-01']
hs_300_news = df_300.values.tolist()

In [118]:
hs_300_news

[['104597500', '2020-07-01', '“焊接专家”联赢激光（688518）：重视研发 技术与市场优势显著', '688518'],
 ['104597749', '2020-07-01', '天合光能拟与TPG集团签署约7亿美元合同', '688599'],
 ['104598568', '2020-07-01', '四川第二家科创板企业上市交易 秦川物联首日盘中涨逾170%', '688528'],
 ['104598603', '2020-07-01', '博瑞医药收监管工作函股价下挫 今年来4收监管文书', '688166'],
 ['104598876', '2020-07-01', '西安“市长特别奖”名单出炉 西部超导等多家上市公司入选', '688122'],
 ['104599063', '2020-07-01', '医疗器械概念午后持续走弱', '688198'],
 ['104599182', '2020-07-01', '秦川物联今日登陆科创板 在智慧燃气领域具备全流程制造工艺', '688528'],
 ['104600023', '2020-07-01', '秦川物联今日登陆科创板 在智能燃气表领域具备全流程制造工艺', '688528'],
 ['104600058', '2020-07-01', '沃尔德：研发实力出众 未来有望进一步实现进口替代', '688028'],
 ['104600359', '2020-07-01', '中微公司获得8217.5万元政府补助', '688012'],
 ['104600378', '2020-07-01', '中微公司：累计收到政府补贴8217.5万元', '688012'],
 ['104600711', '2020-07-02', '海外业务迎来新突破 天合光能拟签署约7亿美元光伏项目合同', '688599'],
 ['104601304', '2020-07-02', '中国通号自主研发轻型智能运行控制系统实现工程应用', '688009'],
 ['104601562', '2020-07-02', '半导体概念股拉升 芯源微涨超13%', '688037'],
 ['104602725', '2020-07-02', '科创板上半年回顾：47股挂牌募资512亿

In [119]:
import pickle
# with open('./data/inno_stock/hs_300_price.pkl', 'wb') as f:
with open('./data/inno_stock/inno_508_news.pkl', 'wb') as f:
    pickle.dump(hs_300_news, f)

# Input Data Construction

## Input Data of Year 2022

In [17]:
com_ids_all = {}
with open('./data/inno_stock/inno_508_code_name', 'r') as f:
    lines = f.readlines()
    for line in lines:
        com_ids_all[line.strip().split('\t')[0]] = line.strip().split('\t')[1]

In [18]:
import pickle
with open('./data/inno_stock/inno_508_price.pkl', 'rb') as f:
    price = pickle.load(f)

In [19]:
com_ids = []
for item in price:
    if item[1] == '2022-01-04':
        com_ids.append(item[0])
        

In [20]:
len(com_ids)

376

In [10]:
idx_2_com_id = {}
idx = 0
for com in com_ids:
    idx_2_com_id[idx] = com
    idx += 1

In [21]:
idx_2_com_name = {}
for idx, com_id in idx_2_com_id.items():
    idx_2_com_name[idx] = com_ids_all[com_id]

In [22]:
idx_2_com_name

{0: '华兴源创',
 1: '睿创微纳',
 2: '天准科技',
 3: '博汇科技',
 4: '容百科技',
 5: '杭可科技',
 6: '光峰科技',
 7: '澜起科技',
 8: '中国通号',
 9: '福光股份',
 10: '新光光电',
 11: '中微公司',
 12: '天臣医疗',
 13: '交控科技',
 14: '心脉医疗',
 15: '绿的谐波',
 16: '乐鑫科技',
 17: '安集科技',
 18: '方邦股份',
 19: '奥福环保',
 20: '瀚川智能',
 21: '安恒信息',
 22: '杰普特',
 23: '洁特生物',
 24: '国盾量子',
 25: '沃尔德',
 26: '南微医学',
 27: '山石网科',
 28: '禾迈股份',
 29: '天宜上佳',
 30: '传音控股',
 31: '芯源微',
 32: '中科通达',
 33: '当虹科技',
 34: '炬芯科技',
 35: '爱博医疗',
 36: '佳华科技',
 37: '龙腾光电',
 38: '莱伯泰科',
 39: '金达莱',
 40: '宝兰德',
 41: '华锐精密',
 42: '云涌科技',
 43: '派能科技',
 44: '凯赛生物',
 45: '航天宏图',
 46: '爱威科技',
 47: '热景生物',
 48: '德林海',
 49: '纵横股份',
 50: '华依科技',
 51: '安旭生物',
 52: '诺泰生物',
 53: '大地熊',
 54: '龙软科技',
 55: '美迪凯',
 56: '映翰通',
 57: '兴图新科',
 58: '盛美上海',
 59: '中望软件',
 60: '三友医疗',
 61: '紫晶存储',
 62: '英科再生',
 63: '虹软科技',
 64: '嘉必优',
 65: '瑞松科技',
 66: '上海谊众',
 67: '爱科科技',
 68: '世华科技',
 69: '福昕软件',
 70: '京源环保',
 71: '博众精工',
 72: '申联生物',
 73: '晶晨股份',
 74: '威胜信息',
 75: '三达膜',
 76: '国力股份',
 77: '诺唯赞',
 78: '金宏

In [78]:
# documents input with date timestamps：company-timestamp-docs
import pickle
with open('./data/inno_stock/inno_508_news.pkl', 'rb') as f:
    news_docs = pickle.load(f)
    

doc_info = []
for idx in range(len(list(idx_2_com_id.keys()))):
    tmp = []
    previous_date = None
    for doc in news_docs:
        if doc[3] == idx_2_com_id[idx] and doc[1] >= '2022-01-01':
            if doc[1] == previous_date:
                tmp[-1][0].append(doc[2])
            else:
                tmp.append([[doc[2]], doc[1]])
                previous_date = doc[1]
                
    doc_info.append(tmp)

In [80]:
len(doc_info)

376

In [81]:
with open('./data/inno_stock/doc_date_input.pkl', 'wb') as f:
    pickle.dump(doc_info, f)

In [24]:
# static graphs input: relation-graphs
import pickle
with open('./data/inno_stock/inno_508_industry.pkl', 'rb') as f:
    industry_graph = pickle.load(f)
    ind_ids = []
    for item in industry_graph:
        if item[1] not in ind_ids:
            ind_ids.append(item[1])
    ind_ids_all = {}
    for item in industry_graph:
        if item[1] not in ind_ids_all:
            ind_ids_all[item[1]] = item[2]

with open('./data/inno_stock/inno_508_invest.pkl', 'rb') as f:
    investment_graph = pickle.load(f)
    inv_ids = []
    for item in investment_graph:
        if item[1] not in inv_ids:
            inv_ids.append(item[1])

idx_2_ind_id = {}
idx = 1000
for ind in ind_ids:
    idx_2_ind_id[idx] = ind
    idx += 1

idx_2_ind_name = {}
for idx, ind_id in idx_2_ind_id.items():
    idx_2_ind_name[idx] = ind_ids_all[ind_id]

idx_2_inv_id = {}
idx = 2000
for inv in inv_ids:
    idx_2_inv_id[idx] = inv
    idx += 1

com_id_2_idx = {}
for k,v in idx_2_com_id.items():
    com_id_2_idx[v] = k

ind_id_2_idx = {}
for k,v in idx_2_ind_id.items():
    ind_id_2_idx[v] = k

inv_id_2_idx = {}
for k,v in idx_2_inv_id.items():
    inv_id_2_idx[v] = k


In [30]:
import networkx as nx

G_ind = nx.Graph()
G_inv = nx.Graph()
G_ind.add_nodes_from(list(idx_2_com_id.keys()))
G_ind.add_nodes_from(list(idx_2_ind_id.keys()))
G_ind.add_nodes_from(list(idx_2_inv_id.keys()))
G_inv.add_nodes_from(list(idx_2_com_id.keys()))
G_inv.add_nodes_from(list(idx_2_ind_id.keys()))
G_inv.add_nodes_from(list(idx_2_inv_id.keys()))
for item in industry_graph:
    if item[0] in list(com_id_2_idx.keys()):
        G_ind.add_edge(com_id_2_idx[item[0]], ind_id_2_idx[item[1]])
for item in investment_graph:
    if item[0] in list(com_id_2_idx.keys()):
        G_inv.add_edge(com_id_2_idx[item[0]], inv_id_2_idx[item[1]])


In [31]:
nx.number_of_edges(G_ind)

393

In [32]:
nx.number_of_edges(G_inv)

646

In [33]:
nx.number_of_nodes(G_inv)

1292

In [35]:
with open('./data/inno_stock/graph_input.pkl', 'wb') as f:
    pickle.dump([G_ind, G_inv, idx_2_com_name, idx_2_ind_name], f)

In [93]:
# indicators input with date timestamps: company-timestamp-indicators
import pickle
with open('./data/inno_stock/inno_508_stats_quarterly.pkl', 'rb') as f:
    finance_indicator = pickle.load(f)

In [94]:
indicator_info = []
for idx in range(len(list(idx_2_com_id.keys()))):
    tmp = []
    for indicator in finance_indicator:
        if indicator[0] == idx_2_com_id[idx] and indicator[4] >= '2021-12-31':
            tmp.append([[indicator[1], indicator[2]], indicator[4]])
    
    indicator_info.append(tmp)
    

In [103]:
with open('./data/inno_stock/indicator_quarter_input.pkl', 'wb') as f:
    pickle.dump(indicator_info, f)

In [79]:
with open('./data/inno_stock/inno_508_price.pkl', 'rb') as f:
    price_indicator = pickle.load(f)

In [80]:
indicator_price_info = []
for idx in range(len(list(idx_2_com_id.keys()))):
    tmp = []
    for indicator in price_indicator:
        if indicator[0] == idx_2_com_id[idx] and indicator[1] >= '2022-01-01':
            tmp.append([[indicator[2], indicator[3]], indicator[1]])
    
    indicator_price_info.append(tmp)

In [81]:
with open('./data/inno_stock/indicator_day_input.pkl', 'wb') as f:
    pickle.dump(indicator_price_info, f)

In [82]:
# daily trend label: 0 for up, 1 for down, 2 for no change(within +-0.1)
trend_daily_label = []
for com_price in indicator_price_info:
    prev_price = None
    tmp = []
    for day_price in com_price:
        current_price = day_price[0][0]
        if prev_price:
            if current_price > prev_price and (current_price - prev_price) >= 0.1:
                tmp.append([0, day_price[1]])
            elif current_price < prev_price and (prev_price - current_price) >= 0.1:
                tmp.append([1, day_price[1]])
            else:
                tmp.append([2, day_price[1]])
            prev_price = current_price
        else:
            prev_price = current_price
            continue
    trend_daily_label.append(tmp)


In [83]:
with open('./data/inno_stock/trend_daily_label.pkl', 'wb') as f:
    pickle.dump(trend_daily_label, f)

In [45]:
wordvec_dict = {}
print('Reading word vectors...')
with open('./data/sgns.financial.bigram-char', encoding='utf-8', errors='ignore') as f:
    for line in f.readlines()[1:]:
        line_list = line.split(' ')
        wordvec_dict[line_list[0]]=[float(item) for item in line_list[1:-1]]


Reading word vectors...


In [47]:
with open('./data/wordvec_dict.pkl', 'wb') as f:
    pickle.dump(wordvec_dict, f)

In [46]:
wordvec_dict['后']

[0.136905,
 -0.286009,
 -0.704455,
 0.351286,
 -0.688953,
 -1.039748,
 -0.113538,
 -1.201819,
 -0.671082,
 -0.326716,
 0.153513,
 -0.501297,
 -0.005518,
 0.058856,
 0.579681,
 -0.423745,
 0.490572,
 -0.689202,
 0.309239,
 -0.346793,
 -0.521018,
 -0.330734,
 -0.088802,
 0.198759,
 -0.941245,
 0.747435,
 -0.143167,
 0.970049,
 -0.822927,
 -1.101076,
 -0.276204,
 -0.270124,
 0.246123,
 0.563766,
 -0.272967,
 -0.714673,
 0.989588,
 0.421837,
 0.099706,
 0.056583,
 -0.199824,
 0.119639,
 0.942607,
 -0.537399,
 -0.291564,
 -0.192206,
 -0.544204,
 -0.569537,
 -0.176537,
 0.167843,
 0.025301,
 -0.060024,
 -0.373628,
 -0.859842,
 -0.052443,
 1.117519,
 -0.10859,
 -0.680084,
 0.126403,
 0.119908,
 0.355746,
 -0.07191,
 -0.528365,
 0.496655,
 -0.011314,
 0.133513,
 0.198244,
 0.243176,
 0.808943,
 -0.283453,
 -0.050419,
 0.154197,
 -0.003884,
 0.130502,
 -0.049045,
 0.150147,
 -0.31321,
 0.831535,
 -0.106715,
 -0.864797,
 -0.156541,
 -0.173222,
 0.134303,
 -0.444971,
 -0.04919,
 0.591824,
 1.0002

In [74]:
import pickle
with open('./data/inno_stock/graph_input.pkl', 'rb') as f:
    graph = pickle.load(f)

In [75]:
import networkx as nx
A = nx.adjacency_matrix(graph[1])

/var/folders/qg/4jdhd9md1t7475h3yhytdxj80000gn/T/ipykernel_17355/4183128813.py:2: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(graph[1])


In [76]:
A.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [77]:
list(graph[1].nodes())

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [1]:
import pickle
with open('./data/inno_stock/doc_date_input.pkl', 'rb') as f:
    doc_input = pickle.load(f)

In [6]:
len(doc_input)

376